In [17]:
import pandas as pd
import numpy as np
%matplotlib inline

In [18]:
#loading the data in
fia_wec_data = pd.read_csv("2012-2022_FIA_WEC_FULL_LAP_DATA_v7.csv", index_col=0)

c:\Users\trist\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (23,26,29,30,31,32,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
#need to map season to string in order to pick the data up.
fia_wec_data['season'] = fia_wec_data['season'].map(str)
fia_wec_data.groupby('season').mean()

,number,driver_number,lap_number,lap_improvement,s1_improvement,s2_improvement,s3_improvement,kph,top_speed,round,lap_time_ms,lap_time_s,driver_stint_no,team_stint_no
season,,,,,,,,,,,,,,
2012,39.932521,0.940329,115.973490,0.010031,0.018555,0.018647,0.018629,175.854385,258.614279,4.033124,162160.609219,162.160609,2.924364,6.983117
2013,49.867881,0.934601,110.071091,0.011619,0.020655,0.020714,0.020743,174.022962,255.483914,3.880146,183888.225948,183.888226,3.011677,7.198721
2014,49.529993,0.925878,113.480249,0.010586,0.019613,0.019613,0.019613,178.038667,268.422846,4.312872,167386.482838,167.386483,3.088030,7.505041
2015,46.819210,0.945960,109.855053,0.010512,0.019772,0.019809,0.019809,175.331826,264.929719,4.253368,166295.950609,166.295951,2.945657,7.114455
2016,44.987310,0.941976,110.682776,0.010291,0.019341,0.019406,0.019406,174.993320,267.091023,4.662002,161091.651051,161.091651,2.888445,7.065400
2017,47.816044,0.942591,113.297124,0.010396,0.019217,0.019217,0.019217,176.561111,269.167103,4.396060,167639.722611,167.639723,3.121173,7.794400
2018-2019,52.492303,1.879858,128.584652,0.009475,0.017871,0.017856,0.017871,187.371539,273.794899,4.699523,199005.723327,199.005723,3.983569,9.962905
2019-2020,53.823907,1.941271,118.077645,0.010156,0.019218,0.019218,0.019218,177.876895,268.123962,5.173450,166250.375897,166.250376,3.235030,8.091294
2021,86.100937,1.985475,127.191145,0.008991,0.017342,0.017342,0.017362,182.586600,273.840004,3.630077,166620.200016,166.620200,3.245949,8.273002


In [20]:
year_filter = ['2012']
data_w_filters = fia_wec_data[
    (fia_wec_data['season'].isin(year_filter))]

In [21]:
#elapsed is fukt so we have to make elapsed_ms
def convert_to_ms(x):
    hour = 0
    try:
        hour, minutes, second = x.split(":")
    except ValueError:
        minutes, second = x.split(":")
    hour_seconds = int(hour)*60*60
    seconds, ms = second.split(".")
    minute_seconds = (int(minutes)*60 + int(seconds) + int(hour_seconds))*1000 
    total_ms = minute_seconds + int(ms)
    return total_ms
data_w_filters['elapsed_s'] = data_w_filters.elapsed.map(convert_to_ms)/1000

#something I didn't think about is gap and position, maybe let's see if we can create one 

data_w_filters_sorted_hour = data_w_filters.sort_values(['round', 'lap_number', 'elapsed_s']).reset_index(drop=True)

C:\Users\trist\AppData\Local\Temp\ipykernel_6024\2160427980.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_w_filters['elapsed_s'] = data_w_filters.elapsed.map(convert_to_ms)/1000


In [22]:
#gets dict for sorted hour by overall
data_w_filters_sorted_hour['key'] = data_w_filters_sorted_hour['season'] + "_" +  data_w_filters_sorted_hour['round'].map(int).map(str) + "_" + data_w_filters_sorted_hour['lap_number'].map(int).map(str) 
wanted_columns = ['key', 'round', 'lap_number']
data_keys = data_w_filters_sorted_hour.groupby('key').mean().reset_index()[wanted_columns].sort_values(['round','lap_number']).reset_index(drop=True)
data_keys['position'] = 0
data_keys_dict = data_keys.to_dict('records')

#gets dict for sorted hour by class
data_w_filters_sorted_hour['key_class'] = data_w_filters_sorted_hour['key'] + "_" +  data_w_filters_sorted_hour['class']
wanted_columns = ['key_class', 'round', 'lap_number']
data_keys_class = data_w_filters_sorted_hour.groupby('key_class').mean().reset_index()[wanted_columns].sort_values(['round','lap_number']).reset_index(drop=True)
data_keys_class['key'] = data_keys_class['key_class']
data_keys_class = data_keys_class.drop(['key_class'], axis=1)
data_keys_class['class_position'] = 0
data_keys_class_dict = data_keys_class.to_dict('records')

def search_key_in_dicts(key, dict):
    for d in dict:
        if d['key'] == key:
            return d
    return None
def update_value_in_dicts(key, dict, col, value):
    dict_key = search_key_in_dicts(key, dict)
    dict_key.update({col : value})
def increment_position(key, dict):
    update_value_in_dicts(key, dict, 'position', search_key_in_dicts(key, dict).get('position') + 1)
    return search_key_in_dicts(key, dict).get('position')
def increment_class_position(key, dict):
    update_value_in_dicts(key, dict, 'class_position', search_key_in_dicts(key, dict).get('class_position') + 1)
    return search_key_in_dicts(key, dict).get('class_position')
    
#loop to run through position, takes a while...
data_w_filters_sorted_hour_r1 = data_w_filters_sorted_hour[data_w_filters_sorted_hour['round'] == 1]
for i in range(0, len(data_w_filters_sorted_hour)):
    x = data_w_filters_sorted_hour.iloc[i]
    increment_position(x['key'], data_keys_dict)
    data_w_filters_sorted_hour.at[i, 'position'] = search_key_in_dicts(x['key'], data_keys_dict).get('position')


In [24]:
data_w_filters_sorted_hour.to_csv('2012_correct_position.csv')

In [26]:
data_w_filters_sorted_hour['team_foreign_key'] = data_w_filters_sorted_hour['season'] + "_" + data_w_filters_sorted_hour['round'].map(int).map(str) \
     + "_" +data_w_filters_sorted_hour['number'].map(int).map(str)
data_w_filters_sorted_hour_last_lap = data_w_filters_sorted_hour.groupby('team_foreign_key').max().reset_index()
data_w_filters_sorted_hour_last_lap['position'] = np.NaN
#data_w_filters_sorted_hour_last_lap['class_position'] = np.NaN
for i in range(0, len(data_w_filters_sorted_hour_last_lap)):
     x = data_w_filters_sorted_hour_last_lap.iloc[i]
     last_lap = data_w_filters_sorted_hour[(data_w_filters_sorted_hour['elapsed_s'] == x['elapsed_s'])&(x['team_foreign_key'] == data_w_filters_sorted_hour['team_foreign_key'])]
     #data_w_filters_sorted_hour_last_lap.at[i, 'class_position'] = last_lap['class_position']
     data_w_filters_sorted_hour_last_lap.at[i, 'position'] = last_lap['position']

C:\Users\trist\AppData\Local\Temp\ipykernel_6024\4219663182.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  data_w_filters_sorted_hour_last_lap = data_w_filters_sorted_hour.groupby('team_foreign_key').max().reset_index()


In [31]:
data_w_filters_sorted_hour_last_lap.sort_values(['season', 'round', 'lap_number'])

,team_foreign_key,number,driver_number,lap_number,lap_time,lap_improvement,s1_improvement,s2_improvement,s3_improvement,kph,...,lap_time_s,engine,driver_stint_no,driver_stint,team_stint_no,team_stint,elapsed_s,key,key_class,position
21,2012_1_59,59.0,1.0,83.0,8:33.288,2.0,2.0,2.0,2.0,179.5,...,513.288,Ferrari,2,Jaime Melo Stint #2,3,Luxury Racing #59 Stint #3,11603.687,2012_1_9,2012_1_9_LMGTE Pro,22.0
8,2012_1_23,23.0,1.0,85.0,3:40.524,2.0,2.0,2.0,2.0,190.1,...,220.524,Nissan,3,Olivier Lombard Stint #3,4,Signatech Nissan #23 Stint #4,11090.867,2012_1_9,2012_1_9_LMP2,9.0
18,2012_1_51,51.0,2.0,214.0,8:18.704,2.0,2.0,2.0,2.0,180.4,...,6529.450,Ferrari,6,Toni Vilander Stint #6,12,AF Corse #51 Stint #9,43278.487,2012_1_99,2012_1_9_LMGTE Pro,26.0
3,2012_1_15,15.0,2.0,233.0,7:39.891,2.0,2.0,2.0,2.0,197.1,...,459.891,Judd,5,Guillaume Moreau Stint #5,10,Oak Racing #15 Stint #9,32482.907,2012_1_99,2012_1_9_LMP1,8.0
19,2012_1_55,55.0,2.0,253.0,6:29.771,2.0,2.0,2.0,2.0,174.1,...,3911.201,Porsche,6,Markus Palttala Stint #6,18,JWA-Avila #55 Stint #9,43364.756,2012_1_99,2012_1_9_LMGTE Am,24.0
20,2012_1_57,57.0,2.0,265.0,4:15.907,2.0,2.0,2.0,2.0,177.8,...,4365.395,Ferrari,4,Tracy Krohn Stint #4,11,Krohn Racing #57 Stint #9,43277.842,2012_1_99,2012_1_9_LMGTE Am,23.0
11,2012_1_28,28.0,2.0,276.0,8:23.288,2.0,2.0,2.0,2.0,186.9,...,2586.886,Nissan,11,Stefan Johansson Stint #8,28,Gulf Racing Middle East #28 Stint #9,43208.974,2012_1_99,2012_1_9_LMP2,22.0
22,2012_1_61,61.0,2.0,283.0,7:33.500,2.0,2.0,2.0,2.0,178.0,...,998.935,Ferrari,6,Rui Aguas Stint #6,15,AF Corse-Waltrip #61 Stint #9,43289.790,2012_1_99,2012_1_9_LMGTE Am,21.0
17,2012_1_50,50.0,2.0,288.0,4:23.201,2.0,2.0,2.0,2.0,176.6,...,263.201,Chevrolet,4,Pedro Lamy Stint #4,11,Larbre Competition #50 Stint #9,41899.629,2012_1_99,2012_1_9_LMGTE Am,15.0
2,2012_1_13,13.0,2.0,291.0,6:53.816,2.0,2.0,2.0,2.0,197.9,...,2272.626,Toyota,9,Jeroen Bleekemolen Stint #9,20,Rebellion Racing #13 Stint #9,43289.678,2012_1_99,2012_1_9_LMP1,19.0


In [13]:
data_w_filters_sorted_hour.to_csv(year_filter[0] + '_data.csv')

In [185]:
#combine the data back together
years = ['2013', '2014', '2015', '2016', '2017', '2018-2019', '2019-2020', '2021', '2022']
full_data = pd.read_csv('2012_data.csv',index_col=0)
for y in years:
    append_data = pd.read_csv(y + '_data.csv',index_col=0)
    full_data = full_data.append(append_data)
full_data = full_data.reset_index(drop=True)

c:\Users\trist\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (29,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\trist\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (11,29,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [186]:
#okay now gap is a harder thing I think. 
#we could record p1 time and then see if we get p2 time? 
#gap by sector would be really hard i think since we'd need time that cross line + other shit.

full_data['season'] = full_data['season'].map(str)

In [187]:
#oh gap is easy. gap to class might be a bit harder?
#okay wait, wait, I think I realized something. we have position. We can calculate gap by organizing everything by lap, position, position in class
full_data_sorted_by_position = full_data.sort_values(['season','round', 'lap_number', 'position']).reset_index(drop=True)
#full_data_sorted_by_position

#gets interval overall
data_shift_minus_one = full_data_sorted_by_position.shift(-1)
data_shift_minus_one['interval'] = data_shift_minus_one['elapsed_ms'] - full_data['elapsed_ms']
negative_interval = data_shift_minus_one[data_shift_minus_one['interval'] < 0]
positive_interval = data_shift_minus_one[data_shift_minus_one['interval'] >= 0]
negative_interval.interval = 0
data_shift_minus_one = negative_interval.append(positive_interval).sort_values(['season','round', 'lap_number', 'position']).reset_index(drop=True)
full_data_sorted_by_position['interval_ms'] = data_shift_minus_one['interval']
full_data_sorted_by_position['interval_ms'] = full_data_sorted_by_position['interval_ms'].shift(1, fill_value=0)
full_data_sorted_by_position['interval'] = full_data_sorted_by_position['interval_ms']/1000
full_data_sorted_by_position

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,driver_stint_no,driver_stint,team_stint_no,team_stint,elapsed_ms,key,key_class,position,class_position,interval_ms
0,2.0,0.0,1.0,1:56.219,0.0,NaN,37.529,0.0,38.945,0.0,...,1,Dindo Capello Stint #1,1,Audi Sport Team Joest #2 Stint #1,116219,2012_1_1,2012_1_1_LMP1,1.0,1.0,0.0
1,1.0,2.0,1.0,1:57.002,0.0,NaN,36.575,0.0,38.786,0.0,...,1,Benoit Tréluyer Stint #1,1,Audi Sport Team Joest #1 Stint #1,117002,2012_1_1,2012_1_1_LMP1,2.0,2.0,783.0
2,3.0,0.0,1.0,1:57.317,0.0,NaN,38.591,0.0,39.306,0.0,...,1,Timo Bernhard Stint #1,1,Audi Sport Team Joest #3 Stint #1,117317,2012_1_1,2012_1_1_LMP1,3.0,3.0,315.0
3,12.0,1.0,1.0,1:59.921,0.0,NaN,39.588,0.0,40.339,0.0,...,1,Neel Jani Stint #1,1,Rebellion Racing #12 Stint #1,119921,2012_1_1,2012_1_1_LMP1,4.0,4.0,2604.0
4,21.0,2.0,1.0,2:00.254,0.0,NaN,39.886,0.0,40.363,0.0,...,1,Jonny Kane Stint #1,1,Strakka Racing #21 Stint #1,120254,2012_1_1,2012_1_1_LMP1,5.0,5.0,333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483173,31.0,3.0,103.0,2:10.406,0.0,NaN,37.172,0.0,1:00.085,0.0,...,3,René RAST Stint #3,8,WRT #31 Stint #8,21697237,2022_2_103,2022_2_103_LMP2,3.0,1.0,38712.0
483174,41.0,2.0,103.0,2:10.761,0.0,NaN,36.841,0.0,1:00.085,0.0,...,3,Ferdinand HABSBURG Stint #3,8,Realteam by WRT #41 Stint #8,21731728,2022_2_103,2022_2_103_LMP2,4.0,2.0,34491.0
483175,38.0,3.0,103.0,2:10.164,0.0,NaN,36.517,0.0,59.479,0.0,...,2,William STEVENS Stint #2,8,JOTA #38 Stint #8,21739623,2022_2_103,2022_2_103_LMP2,5.0,3.0,7895.0
483176,5.0,3.0,103.0,2:10.573,0.0,NaN,36.975,0.0,59.739,0.0,...,3,Felipe NASR Stint #3,7,Team Penske #5 Stint #7,21748662,2022_2_103,2022_2_103_LMP2,6.0,4.0,9039.0


In [189]:
#this gets gap for overall
full_data_sorted_by_position = full_data_sorted_by_position.sort_values(['season','round', 'lap_number', 'position'])
full_data_sorted_by_position['gap'] = full_data_sorted_by_position.groupby('key')['interval'].cumsum()
#ok now that we have interval, let's get gap, this is gap for class
full_data_sorted_by_position = full_data_sorted_by_position.sort_values(['season','round', 'lap_number', 'class', 'class_position']).reset_index(drop=True)
full_data_sorted_by_position['class_interval'] = np.where(full_data_sorted_by_position['class_position']!=1, full_data_sorted_by_position['interval'], 0)
full_data_sorted_by_position['class_gap'] = full_data_sorted_by_position.groupby('key_class')['class_interval'].cumsum()

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,team_stint_no,team_stint,elapsed_ms,key,key_class,position,class_position,interval_ms,interval,gap
0,2.0,0.0,1.0,1:56.219,0.0,NaN,37.529,0.0,38.945,0.0,...,1,Audi Sport Team Joest #2 Stint #1,116219,2012_1_1,2012_1_1_LMP1,1.0,1.0,0.0,0.000,0.000
1,1.0,2.0,1.0,1:57.002,0.0,NaN,36.575,0.0,38.786,0.0,...,1,Audi Sport Team Joest #1 Stint #1,117002,2012_1_1,2012_1_1_LMP1,2.0,2.0,783.0,0.783,0.783
2,3.0,0.0,1.0,1:57.317,0.0,NaN,38.591,0.0,39.306,0.0,...,1,Audi Sport Team Joest #3 Stint #1,117317,2012_1_1,2012_1_1_LMP1,3.0,3.0,315.0,0.315,1.098
3,12.0,1.0,1.0,1:59.921,0.0,NaN,39.588,0.0,40.339,0.0,...,1,Rebellion Racing #12 Stint #1,119921,2012_1_1,2012_1_1_LMP1,4.0,4.0,2604.0,2.604,3.702
4,21.0,2.0,1.0,2:00.254,0.0,NaN,39.886,0.0,40.363,0.0,...,1,Strakka Racing #21 Stint #1,120254,2012_1_1,2012_1_1_LMP1,5.0,5.0,333.0,0.333,4.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483173,31.0,3.0,103.0,2:10.406,0.0,NaN,37.172,0.0,1:00.085,0.0,...,8,WRT #31 Stint #8,21697237,2022_2_103,2022_2_103_LMP2,3.0,1.0,38712.0,38.712,66.185
483174,41.0,2.0,103.0,2:10.761,0.0,NaN,36.841,0.0,1:00.085,0.0,...,8,Realteam by WRT #41 Stint #8,21731728,2022_2_103,2022_2_103_LMP2,4.0,2.0,34491.0,34.491,100.676
483175,38.0,3.0,103.0,2:10.164,0.0,NaN,36.517,0.0,59.479,0.0,...,8,JOTA #38 Stint #8,21739623,2022_2_103,2022_2_103_LMP2,5.0,3.0,7895.0,7.895,108.571
483176,5.0,3.0,103.0,2:10.573,0.0,NaN,36.975,0.0,59.739,0.0,...,7,Team Penske #5 Stint #7,21748662,2022_2_103,2022_2_103_LMP2,6.0,4.0,9039.0,9.039,117.610


In [202]:
full_data_sorted_by_position.drop(['key', 'key_class'], axis=1).to_csv('full_data_with_interval.csv')

In [191]:
full_data_sorted_by_position.to_csv("full_data_with_interval.csv")